In [491]:
from datetime import time

import pandas as pd
import numpy as np
from binance import Client
import matplotlib.pyplot as plt
import sqlalchemy


apikey = "pa3tI72vrqaVH5HZCHCHkxj1OGadlIMy793T0zlVyjj7ocdM8iB82gglvg7DVdan"
apisec = "j4zuHoEzvJbvn6gqVeQjqQW2w2J21frpFMNqRqLny0y9guyme3IwCdHHybfzcfIt"
client = Client(apikey, apisec)

pricesall = pd.DataFrame(client.get_margin_all_pairs())
relevant = pricesall[pricesall['symbol'].str.endswith("USDT")]
relevant = relevant.set_index("symbol")
relevant['N'] = relevant.index
# relevant = pricesall[pricesall['symbol'].str.contains("ABC")]
relevant['N'] = relevant['N'].apply(checkrel)
relevant.dropna(inplace=True)
def getdata(s, i='1h', lookback="400"):
    frame = pd.DataFrame(client.get_historical_klines(s, i, lookback + 'hours UTC'))
    frame = frame.iloc[:, 0:6]
    frame.columns = ['Time', 'Open', "High", 'Low', 'Close', 'Volume']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame


def checkrel(s):
    lis = ["BULL", 'BEAR', 'DOWN', "ABC"]
    addma = True
    for i in lis:
        if i in s:
            addma = False
            break
    if addma:
        return s
    else:
        return None


    
    
def getCoins4Hours():
    
    coinz = {}
    i = 0
    for coin in relevant.index:
        i += 1
        try:
            print(str(i) + " / " + str(relevant.price.count()))
            coinz[coin] = getdata(coin, i="4h", lookback='300')
        except Exception as e:
            relevant.drop(coin)
            print("d" + coin)
    print("Completed")
    for coin in coinz.keys():
        coinz[coin] = pd.DataFrame(coinz[coin])
        coinz[coin].reset_index()
    return coinz

# get the coins over last 10 hours in 15 mins increments
def getcoinsinfo15m():
    i = 0
    coinz15m = {}
    for coin in relevant.index:
        i += 1
        try:
            coinz15m[coin] = getdata(coin, i="15m", lookback='10')
            print(str(i) + " / 350")
        except Exception as e:
            print('dropped ' + coin)
    print("Completed")
    return coinz15m



def gethotandcold4h(coins4h):
    volums = {}
    for coin in coins4h.keys():
        volumes = coins4h[coin]['Volume']
        volums[coin] = int(volumes.mean() * 6)
    volumes = pd.DataFrame(volums.values(), index=volums.keys(), columns=['Volume'])
    print(volumes.sort_values(by="Volume"))
    hotcoins = volumes.sort_values(by="Volume", ascending=False).head(20)
    coldcoins = volumes.sort_values(by="Volume").head(20)
    return [hotcoins, coldcoins]

# Get the coldest and hottest coins over the last 10 hours period
def gethotandcold15m(coins15m):
    volums = {}
    for coin in coins15m.keys():
        volumes = coins15m[coin]['Volume']
        volums[coin] = int(volumes.mean() * 6)
    volumes = pd.DataFrame(volums.values(), index=volums.keys(), columns=['Volume'])
    hotcoins15m = volumes.sort_values(by="Volume", ascending=False).head(20)
    coldcoins15m = volumes.sort_values(by="Volume").head(20)
    return [hotcoins15m, coldcoins15m]

# Get the top gainers and losers over the last 24 hours.
# Works with predefined by binance values, not calculating anything
def getgainandlose24h():
    df = pd.DataFrame(client.get_ticker())
    df = df.loc[df['symbol'].str.endswith("USDT")]
    df = df[['symbol', 'priceChangePercent']]
    df.set_index(df['symbol'], drop=False, inplace=True)
    gainersd = {}
    losersd = {}
    for coin in df.index:
        if float(df.loc[coin].priceChangePercent) > 4.0:
            gainersd[coin] = float(df.loc[coin].priceChangePercent)
        if float(df.loc[coin].priceChangePercent) < -4.0:
            losersd[coin] = float(df.loc[coin].priceChangePercent)
    gainersd = pd.DataFrame(gainersd.values(), index=gainersd.keys(), columns=['priceChangePercent'])
    losersd = pd.DataFrame(losersd.values(), index=losersd.keys(), columns=['priceChangePercent'])
    gainersd.sort_values(by="priceChangePercent", ascending=False, inplace=True)
    losersd.sort_values(by="priceChangePercent", ascending=True, inplace=True)

# Get the general 2 months trend in the market according to BTC, ETH, etc.
def marketaccordingtobigs():
    bigcoins = ["BTCUSDT", "ETHUSDT"]
    indicatorsbigm = 1.0
    changesForWeightedMean = []
    for coin in bigcoins:
        data2 = getdata(coin, i="4h", lookback="360")
        data1 = getdata(coin, i="4h", lookback="180")
        changeMonthly = (data1['Open'].mean() - data2['Open'].mean()) / data1['Open'].mean()
        print(coin + "  -  " + str(changeMonthly))
        changesForWeightedMean.append(float(changeMonthly))
    weightedchange = (changesForWeightedMean[0] * 10 + changesForWeightedMean[1] * 7 + changesForWeightedMean[
        2] * 3) / 20
    indicatorsbigm = indicatorsbigm + weightedchange
    indicatorsbigm = round(indicatorsbigm * 100) - 100
    return indicatorsbigm

# Get the top gainers and losers over the last 3 hours, check if the growth is more than 1.5%
def gainersTime15(z, coins15m):
    potentialgainers = {}
    potentiallosers = {}
    gainerss = {}
    losers = {}
    for coin in coins15m.keys():
        datas = {}
        frame = pd.DataFrame(coins15m[coin].tail(z * 4))
        frame.reset_index(inplace=True)
        for i in frame.index:
            datas[i] = frame.iloc[i].Close
        if (datas[z * 4 - 1] - datas[0]) / datas[0] > 0.015:
            gainerss[coin] = (datas[z * 4 - 1] - datas[0]) / datas[0]
            potentialgainers[coin] = datas
        elif (datas[z * 4 - 1] - datas[0]) / datas[0] < -0.015:
            losers[coin] = (datas[z * 4 - 1] - datas[0]) / datas[0]
            potentiallosers[coin] = datas
    gainerss = pd.DataFrame(gainerss.values(), columns=['Percent Change'], index=gainerss.keys())
    losers = pd.DataFrame(losers.values(), columns=['Percent Change'], index=losers.keys())
    losers.sort_values(by="Percent Change", ascending=True, inplace=True)
    gainerss.sort_values(by='Percent Change', ascending=False, inplace=True)
    return [gainerss, losers]


# If the volume is very big, while the coin itself is not as popular we are going to keep notice of it
# to see the populaity spikes, we will look for increases in volume compared to average volume of that coin
# we have our monthly data in 15minutes candles, and we will first sum 96 15 minute datapoints = 1 day,
# and then find a mean for one day volume, and compare it to that of today. If today is rather high volume
# we will let the user know
def volchangers(coins4h):
    potentials = {}

    for coin in coins4h:
        data = pd.DataFrame(coins4h[coin].Volume)
        l = data.count()
        mean = data.mean()
        if data.tail(4).Volume.mean() / mean.Volume > 1.2:
            potentials[coin] = data.tail(4).Volume.mean() / mean.Volume

    volchangers = pd.DataFrame(potentials.values(), columns=["Percent change in volume"], index=potentials.keys())
    volchangers.sort_values(by="Percent change in volume", inplace=True, ascending=False)
    return volchangers



engine = sqlalchemy.create_engine('sqlite:///hotcoins.db')
#hotcoins15m.to_sql('hot15m',engine,if_exists='replace',index=True)
#coldcoins15m.to_sql('cold15m',engine,if_exists='replace',index=True)
#gainersd.to_sql('gain1d',engine,if_exists='replace',index=True)
#losersd.to_sql('lose1d',engine,if_exists='replace',index=True)
#gainers3h.to_sql('gain3h',engine,if_exists='replace',index=True)
#losers3h.to_sql('lose3h',engine,if_exists='replace',index=True)

def getCoins1Hour():
    pricesall = pd.DataFrame(client.get_all_tickers())
    relevant = pricesall[pricesall['symbol'].str.endswith("USDT")]
    relevant = relevant.set_index("symbol")
    relevant['N'] = relevant.index
    # relevant = pricesall[pricesall['symbol'].str.contains("ABC")]
    relevant['N'] = relevant['N'].apply(checkrel)
    relevant.dropna(inplace=True)
    coinz = {}
    i = 0
    for coin in relevant.index:
        i += 1
        try:
            if i%10 == 0:
                print(str(i) + " / " + str(relevant.price.count()))
            coinz[coin] = getdata(coin, i="1h", lookback='240')
        except Exception as e:
            relevant.drop(coin)
            print("d" + coin)
    print("Completed")
    for coin in coinz.keys():
        coinz[coin] = pd.DataFrame(coinz[coin])
        coinz[coin].reset_index()
    return coinz

def getCoins5minutes(coinslist):
    coinz = {}
    i = 0
    for coin in coinslist:
        i += 1
        try:
            print(str(i) + " / " + str(len(coinslist)))
            coinz[coin] = getdata(coin, i="5m", lookback='4')
        except Exception as e:
            relevant.drop(coin)
            print("d" + coin)
    print("Completed")
    for coin in coinz.keys():
        coinz[coin] = pd.DataFrame(coinz[coin])
        coinz[coin].reset_index()
    return coinz
def getCoins1minute(coinslist):
    coinz = {}
    i = 0
    for coin in coinslist:
        i += 1
        try:
            print(str(i) + " / " + str(relevant.price.count()))
            coinz[coin] = getdata(coin, i="1m", lookback='4')
        except Exception as e:
            relevant.drop(coin)
            print("d" + coin)
    print("Completed")
    for coin in coinz.keys():
        coinz[coin] = pd.DataFrame(coinz[coin])
        coinz[coin].reset_index()
    return coinz
def getCoins1minutegen():
    coinz = {}
    i = 0
    for coin in relevant.index:
        i += 1
        try:
            #print(str(i) + " / ")
            coinz[coin] = getdata(coin, i="1m", lookback='1')
        except Exception as e:
            relevant.drop(coin)
            print("d" + coin + "     " + str(e))
    print("Completed")
    for coin in coinz.keys():
        coinz[coin] = pd.DataFrame(coinz[coin])
        coinz[coin].reset_index()
    return coinz



def checkPump1(data15):
    #first check those who rose by more than 1.5% over the last 30 mins.
    #we will work with them
    morethan2in15 = {}
    for coin in data15.keys():
        coinz15 = data15[coin].iloc[147:].Close.values
        if coinz15[11]/coinz15[9] > 1.015:
            morethan2in15[coin] = coinz15[11]/coinz15[10] - 1.0
    #then we want to see the overall trend during last 30 hours, for which we will need 120 15minute increments
    stablecoins=[]
    for coin in morethan2in15.keys():
        coinz15 = data15[coin].iloc[39:].Close.values
        #if in any subsequent 4 hour periods, there was NO big pump, which is rise by 7 SEVEN PERCENT, we want to continue
                                                    
            
            
                                #######EVERYTHING BELOW IS DEACTIVATED
        state = ''
        index = 0
        print(coin + " checking:")
        while True:
            try:
                fourhourschange = coinz15[index+16]/coinz15[index] - 1.0
                if fourhourschange > 0.07:
                    state= state + 'rise'
                    print(coin + ' pumped around ' +  str(15 *(120 - index)//60) + ' h ago')
                    index+=8
                if fourhourschange < -0.07:
                    state= state + 'fall'
                    print(coin + ' fell around ' +  str(15 *(120 - index)//60) + ' h ago')
                    index+=8
            except Exception as e:
                break
            index+=1
                                #############EVERYTHING ABOVE IS DEACTIVATED
        #if state=='':
        stablecoins.append(coin)
    print(stablecoins)
    return stablecoins


def checkpump2(stables):
    df = getCoins5minutes(stables)
    pump={}
    fall={}
    for coin in stables:
        closes=df[coin].iloc[36:].Close.values
        for i in range(len(closes)):
            closes[i] = float(closes[i])
        percentimp = 0.001

#  1  check if there is a rise of 5% last hour(12 * 5 minute periods ago), and if so, then 
        #if closes[0]/closes[-1] >= (1+percentimp) | closes[0]/closes[-2] >= (1+percentimp) | closes[1]/closes[-1]>(1+percentimp) | closes[1]/closes[-2] >(1+percentimp):
        if closes[0]/closes[-1] >= (1+percentimp):    
            #print(coin + " pumped " + str(closes[0]/closes[-1] - 1) + "% during last hour")
            pump[coin] = closes
        #elif closes[0]/closes[-1] <= (1-percentimp) | closes[0]/closes[-2] <= (1-percentimp) | closes[1]/closes[-1] < (1-percentimp) | closes[1]/closes[-2] < (1-percentimp):
        elif closes[0]/closes[-1] <= (1-percentimp):
            #print(coin + " fell " + str(closes[0]/closes[-1] - 1) + "% during last hour")
            fall[coin] = closes
            
        print(coin)
#  2  check if the last 25 min is also positive overall, and if so invest
    coinsfinalp={}
    for coin in pump.keys():
        poss = 0
        for i in range(7,12):
            if (pump[coin][i]/pump[coin][i-1]>=1):
                poss+=1
        if poss >= 3:
           # print(coin + ' overall positive on second test with ' + str(poss) + " positive increments")
            coinsfinalp[coin] = poss
            
            
        #and for failers
    coinsfinalf={}
    for coin in fall.keys():
        poss = 0
        for i in range(7,12):
            if (fall[coin][i]/fall[coin][i-1]<=1):
                poss+=1
        if poss >= 3:
            #print(coin + ' overall negative on second test with ' + str(poss) + " negative increments")
            coinsfinalf[coin] = poss
    print(coinsfinalf)
    print(coinsfinalp)
    tobuy = pd.DataFrame(coinsfinalp.values(), index=coinsfinalp.keys(), columns=['positives'])
    print(tobuy.sort_values(by='positives'))
    tosell = pd.DataFrame(coinsfinalf.values(), index=coinsfinalf.keys(), columns=['negatives'])
    print(tosell.sort_values(by='negatives'))
#  3  put a tolerance check, where as soon as we are to lose 2% from current profit we sell
        
        #    1  
        
        
        
marginall = pd.DataFrame(client.get_margin_all_pairs())
rel = marginall[marginall['symbol'].str.endswith("USDT")]
rel = rel.set_index("symbol")
rel['N'] = rel.index
rel['N'] = rel['N'].apply(checkrel)
rel.dropna(inplace=True)

def checkStableRise2percent(onecoin1m):
    prcntimp = 1.002
    data=onecoin1m.tail(20)
    print(data)
    poses = 0
    prev = False
    prevprev = False
    for i in range(19):
        if (data[i+1]/data[i] > prcntimp):
            poses+=1
        elif data[i+1]/data[i] > -prcntimp:
            if prevprev == True:
                print(coin + " 2 reds in a row")
                prevprev=False
                prev = False
            if prev == True:
                prevprev=True
            prev = True
        else:
            print(coin + " red bigger than " + str(prcntimp))
    return poses
# yaniki, if a coin moves stably, stably being 7 out of 10 minute candles being the same color,
# and normal in size, say 0.2% each, and red candles are not bigger than 0.15% each,

# and if the red candles are not adjacent
# then we want to buy a coin and hold for 5% profit with tolerance of 7%

def gainersTime1(z):
    potentialgainers = {}
    potentiallosers = {}
    gainerss = {}
    losers = {}
    percentimp = 0.01
    coinz = {}
    coins02pctcheck={}
    i = 0
    
    for coin in rel.index:
        i += 1
        try:
            #print(str(i) + " / ")
            coinz[coin] = getdata(coin, i="1m", lookback='1')
            coinz[coin] = pd.DataFrame(coinz[coin])
            coinz[coin].reset_index()
            datas = {}
            frame = pd.DataFrame(coinz[coin].tail(z))
            frame.reset_index(inplace=True)
        #0.2% check
            passes = checkStableRise2percent(coinz[coin].Open.values(), coin)
            coins02pctcheck[coin] = passes
            for i in frame.index:
                datas[i] = frame.iloc[i].Close
                
            if (datas[z - 1] - datas[0]) / datas[0] > percentimp:
                gainerss[coin] = (datas[z - 1] - datas[0]) / datas[0]
                potentialgainers[coin] = datas
                print(coin + '  Gained  ' + str((datas[z - 1] - datas[0]) / datas[0]))
                print(passes)

            elif (datas[z  - 1] - datas[0]) / datas[0] < -percentimp:
                losers[coin] = (datas[z - 1] - datas[0]) / datas[0]
                potentiallosers[coin] = datas
                print(coin + '  Lost  ' + str((datas[z - 1] - datas[0]) / datas[0]))
            

        except Exception as e:
            rel.drop(coin, inplace=True)
            
    print("Completed")
        

        
    gainerss = pd.DataFrame(gainerss.values(), columns=['Percent Change'], index=gainerss.keys())
    losers = pd.DataFrame(losers.values(), columns=['Percent Change'], index=losers.keys())
    losers.sort_values(by="Percent Change", ascending=True, inplace=True)
    gainerss.sort_values(by='Percent Change', ascending=False, inplace=True)
    for i in gainerss.index:
        print(i + " " + coins02pctcheck[i])
    return [gainerss, losers]

#Hopping on something that rises stably, catching its small pump (1-2%) and putting a feature, taking tolerance profit 3 %is 

#But how can you catch it? Track the motion, for example People coins graph on 5/23 at night was fallig, 
#and the best indicator for that would be something like the fact that most of 1 or 5 minute candles were red, 
#thus the close was lower than open. Moreover, the percentage changes of each candle were insignificant. This
#is equally important. Thus, i can track coins with obvios trends and create futures on them, wait for profits 
#stop loss is triggered, being -5%, and then, after say 3% profit, i can put a tolerance of 6% and wait for the
#trend to realize fully. This way i will be able to track steady coins like people at ~4 am 23rd of may





# if the coin rose more than 7% in the last hour or less, but has only started 
# rising this hour, we want to buy it and hold with a tolerance of 3% loss every tick.
#
# started rising means that the increase in the value before this was just around 2%

# 1    Thus, first we will need a 15 min tracker, and first we will check for the rise in the last 30 minutes, then next 
# 1        we will check for 45 minutes and then an 60 minutes and then 75 minutes and then 90 minutes

# 1       as soon as we will see the rise by or more than 7%, we want to buy the coin.

# if the coin rose more than 20% in one hour, then we will want to short it untill we get 3% profit
# if the coin has rose over 4% in 15minutes,we want to buy and wait for 2% profit.
# if the coin is a top gainer, we want to see when its volume becomes selling volume, and if the coin is being sold heavily,
# as heavily as was bought, we short it
        

In [492]:
coins1m=gainersTime1(30)
coins1m

Completed


[Empty DataFrame
 Columns: [Percent Change]
 Index: [],
 Empty DataFrame
 Columns: [Percent Change]
 Index: []]